In [1]:
import asyncio
import datetime as dt
import math
from typing import Literal

import matplotlib.pyplot as plt
import mplfinance as mpf
import numpy as np
import pandas as pd
import pandas_market_calendars as mcal
import plotly.graph_objects as go
import polars as pl
from dash import Dash, dcc, html
from plotly.subplots import make_subplots

nse = mcal.get_calendar("NSE")

In [2]:
pd.set_option("display.max_rows", 25_000)
pd.set_option("display.max_columns", 500)
pl.Config.set_tbl_cols(500)
pl.Config.set_tbl_rows(10_000)

pd.options.display.float_format = "{:.4f}".format

import sys

sys.path.append("..")
from tooling.enums import AssetClass, Index, Spot, StrikeSpread
from tooling.fetch import fetch_option_data, fetch_spot_data
from tooling.filter import find_atm, option_tool

from fetching_from_local_db.enums import AssetClass, Index, StrikeSpread
from fetching_from_local_db.fetch_from_db import _fetch_batch, fetch_data, fetch_spot_data

In [3]:
async def get_expiry(f_today):

    if (f_today <= dt.date(2024, 1, 25)) and (f_today >= dt.date(2024, 1, 18)):
        f_expiry = dt.date(2024, 1, 25)
    elif (f_today <= dt.date(2024, 1, 31)) and (f_today >= dt.date(2024, 1, 26)):
        f_expiry = dt.date(2024, 1, 31)
    elif (f_today <= dt.date(2024, 2, 22)) and (f_today >= dt.date(2024, 2, 29)):
        f_expiry = dt.date(2024, 2, 29)
    elif (f_today <= dt.date(2024, 3, 25)) and (f_today >= dt.date(2024, 3, 27)):
        f_expiry = dt.date(2024, 2, 27)
    elif f_today < dt.date(2023, 9, 1):
        days_to_thursday = (3 - f_today.weekday()) % 7
        nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
        f_expiry = nearest_thursday
        if nse.valid_days(start_date=nearest_thursday, end_date=nearest_thursday).empty:
            f_expiry = nearest_thursday - dt.timedelta(days=1)
    elif f_today >= dt.date(2023, 9, 1):
        if f_today.day < 24:
            days_to_wednesday = (2 - f_today.weekday()) % 7
            nearest_wednesday = f_today + dt.timedelta(days=days_to_wednesday)
            f_expiry = nearest_wednesday
            if nse.valid_days(
                start_date=nearest_wednesday, end_date=nearest_wednesday
            ).empty:
                f_expiry = nearest_wednesday - dt.timedelta(days=1)
        else:
            days_to_thursday = (3 - f_today.weekday()) % 7
            nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
            f_expiry = nearest_thursday
            if nse.valid_days(
                start_date=nearest_thursday, end_date=nearest_thursday
            ).empty:
                f_expiry = nearest_thursday - dt.timedelta(days=1)
    return f_expiry


async def get_expiry_nifty(f_today):

    days_to_thursday = (3 - f_today.weekday()) % 7
    nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
    f_expiry = nearest_thursday
    if nse.valid_days(start_date=nearest_thursday, end_date=nearest_thursday).empty:
        f_expiry = nearest_thursday - dt.timedelta(days=1)
    return f_expiry

async def get_expiry_sensex(f_today):

    days_to_thursday = (4 - f_today.weekday()) % 7
    nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
    f_expiry = nearest_thursday
    if nse.valid_days(start_date=nearest_thursday, end_date=nearest_thursday).empty:
        f_expiry = nearest_thursday - dt.timedelta(days=1)
    return f_expiry


async def get_option_contract_name(symbol, strike, expiry, opt_type):
    temp = "0"
    mth = expiry.month

    if (expiry + dt.timedelta(days=7)).month != expiry.month:
        date_string = expiry.strftime("%y%b").upper()
        return f"{symbol}{date_string}{strike}{opt_type}"
    else:
        if expiry.day <= 9:
            date_string = f"{expiry.year - 2000}{mth}{temp}{expiry.day}"
        else:
            date_string = f"{expiry.year - 2000}{mth}{expiry.day}"
        return f"{symbol}{date_string}{strike}{opt_type}"


def get_option_contract_name2(symbol, strike, expiry, opt_type):
    temp = "0"
    mth = expiry.month

    if (expiry + dt.timedelta(days=7)).month != expiry.month:
        date_string = expiry.strftime("%y%b").upper()
        return f"{symbol}{date_string}{strike}{opt_type}"
    else:
        if expiry.day <= 9:
            date_string = f"{expiry.year - 2000}{mth}{temp}{expiry.day}"
        else:
            date_string = f"{expiry.year - 2000}{mth}{expiry.day}"
        return f"{symbol}{date_string}{strike}{opt_type}"

In [4]:
# bnf_pandas = pd.read_csv('../data/nifty_1hr_tv (2).csv')
# bnf_pandas = pd.read_csv('../data/bnf_1hr_tv.csv')
# bnf_pandas = pd.read_csv('../data/midcp_select_1hr_tv.csv')
# bnf_pandas = pd.read_csv('../data/sensex_1hr_tv.csv')
# bnf_pandas = pd.read_csv('../data/crude_4hr_tv.csv')
# bnf_pandas = pd.read_csv('../data/gold_4hr_tv.csv')
# bnf_1min = pd.read_csv('../data/bnf_min.csv')
bnf_1min = pd.read_csv("../data/nifty_min_2019-2024 (1).csv")
bnf_1min.columns = ['index', 'datetime', 'o', 'h', 'l', 'c', 'v']
bnf_1min.head()

,index,datetime,o,h,l,c,v
0,nifty,2017-01-02 09:15:00,8210.1000,8211.7000,8189.0000,8189.5500,0
1,nifty,2017-01-02 09:16:00,8188.7500,8193.9500,8188.7500,8189.9500,0
2,nifty,2017-01-02 09:17:00,8190.1500,8190.7500,8173.7000,8173.7000,0
3,nifty,2017-01-02 09:18:00,8173.3500,8177.5500,8169.1500,8177.5500,0
4,nifty,2017-01-02 09:19:00,8177.8500,8178.1500,8173.4500,8174.4000,0


In [5]:
bnf_1min["datetime"] = pd.to_datetime(bnf_1min["datetime"]).dt.tz_localize(None)
# bnf_1min = bnf_1min[((bnf_1min['datetime'].dt.year == 2020) & (bnf_1min['datetime'].dt.month == 4))]
bnf_1min = bnf_1min[
    (bnf_1min["datetime"].dt.year >= 2017) & (bnf_1min["datetime"].dt.year <= 2024)
]

In [6]:
def resample(
    data: pl.DataFrame, timeframe, offset: dt.timedelta | None = None
) -> pl.DataFrame:
    return (
        data.set_sorted("datetime")
        .group_by_dynamic(
            index_column="datetime",
            every=timeframe,
            period=timeframe,
            label="left",
            offset=offset,
        )
        .agg(
            [
                pl.col("open").first().alias("open"),
                pl.col("high").max().alias("high"),
                pl.col("low").min().alias("low"),
                pl.col("close").last().alias("close"),
                # pl.col("volume").sum().alias("volume"),
            ]
        )
    )


# ohlc_resampled = resample(pl.DataFrame(bnf_1min), '7d', pd.Timedelta(days=4))
# ohlc_resampled

In [7]:
# HL -> High Level
# LL -> Low Level
# PDH -> Previous Day High
# PDL -> Previous Day Low

# Variation 1:
## Entry : HL / LL , SL : PDH / PDL , Condition: Spot breaking HL/LL

# Variation 2:
## Entry : ATM at 9:15 open, SL: PDH / PDL , Condition : Spot breaking HL/LL

# Variation 3:
## Entry: ATM at 9:15 open, SL : HL, LL

# Variation 4:
## Entry : ATM , SL : x% above Entry Premium

In [8]:
async def find_breakouts_v1(df, daily_ranges, pct):

    results = []
    multiplier_to_range = pct

    portfolio_value = 90_00_000
    index_leverage = 9

    index_name = "NIFTY"
    index_str_for_opt = "nifty"

    daily_ranges["datetime"] = pd.to_datetime(daily_ranges["datetime"])
    # weekly_ranges = weekly_ranges[:-2]
    # print(weekly_ranges)

    dates_to_avoid = [
        # dt.date(2017, 1, 23),
        # dt.date(2017, 3, 13),
        # dt.date(2017, 4, 10),
        # dt.date(2017, 12, 11),
        # dt.date(2019, 3, 25),
        # dt.date(2019, 4, 1),
        # dt.date(2019, 9, 9),
        # dt.date(2020, 4, 6),
        # dt.date(2020, 10, 26),
    ]

    for i in range(1, len(daily_ranges)):
        previous_day = daily_ranges.iloc[i - 1]
        current_day = daily_ranges.iloc[i]
        # print(current_week)
        # print("Current day : ", current_day["datetime"].date())
        if current_day["datetime"].date() in dates_to_avoid:
            print("Date Avoided")
            continue
        previous_day_high = daily_ranges.iloc[i - 1]["high"]
        previous_day_low = daily_ranges.iloc[i - 1]["low"]
        previous_day_close = daily_ranges.iloc[i - 1]["close"]

        current_day_open = daily_ranges.iloc[i]["open"]
        # print(previous_day_high, previous_day_low, current_day_open)
        # Filter the 1-minute data for the current week
        # current_week_data = df[(df.index > weekly_ranges.index[i-1]) & (df.index <= current_week)]
        current_day_data = df.loc[df["datetime"].dt.date == current_day["datetime"].date()]
        # print("Current Day Data:\n", current_day_data)

        daily_range = previous_day_high - previous_day_low
        addition_range = multiplier_to_range * daily_range

        # Check for breakouts
        high_level = current_day_data["open"].iloc[0] + addition_range
        low_level = current_day_data["open"].iloc[0] - addition_range

        # high_level = current_week_data['open'].iloc[0] + addition_range
        # low_level = current_week_data['open'].iloc[0] - addition_range
        # print(current_week)
        # print("High Level:\n", high_level, "\nLow Level:\n", low_level)

        breakout_high = current_day_data[current_day_data["high"] >= high_level]
        breakout_low = current_day_data[current_day_data["low"] <= low_level]

        # break

        if not breakout_high.empty:
            # print('Breakout High')
            breakout_high_time = breakout_high.iloc[0]["datetime"]
            # print('High Break At :', breakout_high_time)
            atm_strike = int(round(low_level / 50) * 50)
            # exit_date = current_week['datetime'].date() + dt.timedelta(days=5)
            expiry = await get_expiry_nifty(current_day["datetime"].date())
            dte = (expiry - breakout_high_time.date()).days
            contract = await get_option_contract_name(
                symbol=index_name,
                strike=atm_strike,
                expiry=expiry,
                opt_type="PE",
            )
            pe_df = await fetch_data(
                index=index_str_for_opt,
                start_date=breakout_high_time.date(),
                end_date=breakout_high_time.date(),
                start_time=breakout_high_time.time(),
                end_time=dt.time(15, 20),
                expiry=expiry,
                strike=atm_strike,
                asset_class="P",
            )
            if not isinstance(pe_df, str) and pe_df is not None:
                pe_df = pe_df.to_pandas()
                entry_price = pe_df.iloc[0]["c"]
            else:
                entry_price = float("nan")

            # Exit Scenario
            current_day_data_after_entry = current_day_data[
                (current_day_data["datetime"] > breakout_high_time)
                & (current_day_data["datetime"].dt.time <= dt.time(15, 30))
            ]
            # print(current_week_data_after_entry)
            low_breach = current_day_data_after_entry[
                current_day_data_after_entry["low"] < previous_day_low
            ]
            # print(low_breach)
            if len(low_breach) != 0:
                exit_time = low_breach.iloc[0]["datetime"]
                # print(exit_time)
                remark = "SL Hit"
                # print(remark, exit_time)
                # print(pe_df)
                if not math.isnan(entry_price):
                    # print((pe_df[(pe_df['datetime'].dt.date == exit_time.date()) & (pe_df['datetime'].dt.hour >= exit_time.hour) & (pe_df['datetime'].dt.minute >= exit_time.minute)]).head(3))
                    filtered_df = pe_df.loc[
                        (pe_df["datetime"].dt.date >= exit_time.date())
                        & (pe_df["datetime"].dt.hour >= exit_time.hour)
                        & (pe_df["datetime"].dt.minute >= exit_time.minute),
                        "c"]
                    exit_price = filtered_df.iloc[0] if not filtered_df.empty else float('nan')

                else:
                    exit_price = float("nan")
                    # print(exit_price)
            else:
                exit_time = dt.datetime.combine(
                    current_day["datetime"].date(), dt.time(15, 30)
                )
                remark = "EOD Closing"
                # print(remark, exit_time)
                if not math.isnan(entry_price):
                    exit_price = pe_df.iloc[-1]["c"]
                else:
                    exit_price = float("nan")
                    # print(exit_price)
            # print(exit_time)
            # print(exit_price)
            # print(contract)
            qty = portfolio_value * index_leverage / atm_strike
            slippage = 0.01 * (entry_price + exit_price)
            pnl = qty * (entry_price - exit_price)
            final_pnl = qty * (entry_price - exit_price - slippage)
            unit = {
                "Previous Day": previous_day["datetime"].date(),
                "Day High": previous_day_high,
                "Day Low": previous_day_low,
                "Daily Range": daily_range,
                "Current Day Open": current_day_open,
                "Multiplier to Range": f"{int(multiplier_to_range * 100)}%",
                "Entry Level Long": high_level,
                "Entry Level Short": low_level,
                "Break Type": "high",
                "Strike": atm_strike,
                "Expiry": expiry,
                "DTE": dte,
                "Contract": contract,
                "Option Type": contract[-2:],
                "Entry Time": breakout_high_time,
                "Entry Price": entry_price,
                "Exit Time": exit_time,
                "Exit Price": exit_price,
                "Remark": remark,
                "Points": entry_price - exit_price,
                "Slippage": slippage,
                "Points w cs": (entry_price - exit_price) - slippage,
                "Qty": qty,
                "PnL": pnl,
                "PnL w cs": final_pnl,
                "ROI%": (pnl * 100 / portfolio_value),
                "ROI% w cs": (final_pnl * 100 / portfolio_value),
            }
            # print(unit)
            results.append(unit)

        if not breakout_low.empty:
            # print('Breakout Low')
            breakout_low_time = breakout_low.iloc[0]["datetime"]
            # print('Low Break At :', breakout_low_time)
            atm_strike = int(round(high_level / 50) * 50)
            # exit_date = current_week['datetime'].date() + dt.timedelta(days=5)
            expiry = await get_expiry_nifty(current_day["datetime"].date())
            dte = (expiry - breakout_low_time.date()).days
            contract = await get_option_contract_name(
                symbol=index_name,
                strike=atm_strike,
                expiry=expiry,
                opt_type="CE",
            )
            ce_df = await fetch_data(
                index=index_str_for_opt,
                start_date=breakout_low_time.date(),
                end_date=breakout_low_time.date(),
                start_time=breakout_low_time.time(),
                end_time=dt.time(15, 20),
                expiry=expiry,
                strike=atm_strike,
                asset_class="C",
            )
            # print(len(ce_df))
            if not isinstance(ce_df, str) and ce_df is not None:
                ce_df = ce_df.to_pandas()
                entry_price = ce_df.iloc[0]["c"]
            else:
                entry_price = float("nan")

            # Exit Scenario
            current_day_data_after_entry = current_day_data[
                (current_day_data["datetime"] > breakout_low_time)
                & (current_day_data["datetime"].dt.time <= dt.time(15, 30))
            ]
            # print(current_week_data_after_entry)
            high_breach = current_day_data_after_entry[
                current_day_data_after_entry["high"] > previous_day_high
            ]
            # print(high_breach)
            if len(high_breach) != 0:
                exit_time = high_breach.iloc[0]["datetime"]
                # print(exit_time)
                remark = "SL Hit"
                # print(remark, exit_time)
                if not math.isnan(entry_price):
                    # print(exit_time.date(), exit_time.hour, exit_time.minute)
                    # print((ce_df[(ce_df['datetime'].dt.date == exit_time.date()) & (ce_df['datetime'].dt.hour >= exit_time.hour) & (ce_df['datetime'].dt.minute >= exit_time.minute)]).head(3))
                    filtered_df = ce_df.loc[
                        (ce_df["datetime"].dt.date >= exit_time.date())
                        & (ce_df["datetime"].dt.hour >= exit_time.hour)
                        & (ce_df["datetime"].dt.minute >= exit_time.minute),
                        "c"]
                    exit_price = filtered_df.iloc[0] if not filtered_df.empty else float('nan') 
                else:
                    exit_price = float("nan")
                    # print(exit_price)
            else:
                exit_time = dt.datetime.combine(
                    current_day["datetime"].date(), dt.time(15, 30)
                )
                remark = "EOD Closing"
                # print(remark, exit_time)
                if not math.isnan(entry_price):
                    exit_price = ce_df.iloc[-1]["c"]
                else:
                    exit_price = float("nan")
                    # print(exit_price)
            # print(exit_time)
            # print(exit_price)
            # print(contract)

            qty = portfolio_value * index_leverage / atm_strike
            slippage = 0.01 * (entry_price + exit_price)
            pnl = qty * (entry_price - exit_price)
            final_pnl = qty * (entry_price - exit_price - slippage)
            unit = {
                "Previous Day": previous_day["datetime"].date(),
                "Day High": previous_day_high,
                "Day Low": previous_day_low,
                "Daily Range": daily_range,
                "Current Day Open": current_day_open,
                "Multiplier to Range": f"{int(multiplier_to_range * 100)}%",
                "Entry Level Long": high_level,
                "Entry Level Short": low_level,
                "Break Type": "low",
                "Strike": atm_strike,
                "Expiry": expiry,
                "DTE": dte,
                "Contract": contract,
                "Option Type": contract[-2:],
                "Entry Time": breakout_low_time,
                "Entry Price": entry_price,
                "Exit Time": exit_time,
                "Exit Price": exit_price,
                "Remark": remark,
                "Points": entry_price - exit_price,
                "Slippage": slippage,
                "Points w cs": (entry_price - exit_price) - slippage,
                "Qty": qty,
                "PnL": pnl,
                "PnL w cs": final_pnl,
                "ROI%": (pnl * 100 / portfolio_value),
                "ROI% w cs": (final_pnl * 100 / portfolio_value),
            }
            # print(unit)
            results.append(unit)

    # print(results)
    return results

In [9]:
async def find_breakouts_v2(df, daily_ranges, pct):

    results = []
    multiplier_to_range = pct
    max_high = float("nan")
    max_low = float("nan")

    portfolio_value = 90_00_000
    index_leverage = 9

    index_name = "NIFTY"
    index_str_for_opt = "nifty"

    daily_ranges["datetime"] = pd.to_datetime(daily_ranges["datetime"])
    # weekly_ranges = weekly_ranges[:-2]
    # print(weekly_ranges)

    dates_to_avoid = [
        # dt.date(2017, 1, 23),
        # dt.date(2017, 3, 13),
        # dt.date(2017, 4, 10),
        # dt.date(2017, 12, 11),
        # dt.date(2019, 3, 25),
        # dt.date(2019, 4, 1),
        # dt.date(2019, 9, 9),
        # dt.date(2020, 4, 6),
        # dt.date(2020, 10, 26),
    ]

    for i in range(1, len(daily_ranges)):
        previous_day = daily_ranges.iloc[i - 1]
        current_day = daily_ranges.iloc[i]
        # print(current_week)
        print("Current day : ", current_day["datetime"].date())
        if current_day["datetime"].date() in dates_to_avoid:
            print("Date Avoided")
            continue
        previous_day_high = daily_ranges.iloc[i - 1]["high"]
        previous_day_low = daily_ranges.iloc[i - 1]["low"]
        previous_day_close = daily_ranges.iloc[i - 1]["close"]

        current_day_open = daily_ranges.iloc[i]["open"]
        # print(previous_day_high, previous_day_low, current_day_open)
        # Filter the 1-minute data for the current week
        # current_week_data = df[(df.index > weekly_ranges.index[i-1]) & (df.index <= current_week)]
        current_day_data = df.loc[df["datetime"].dt.date == current_day["datetime"].date()]
        # print("Current Day Data:\n", current_day_data)

        daily_range = previous_day_high - previous_day_low
        addition_range = multiplier_to_range * daily_range

        # Check for breakouts
        high_level = current_day_data["open"].iloc[0] + addition_range
        low_level = current_day_data["open"].iloc[0] - addition_range

        # high_level = current_week_data['open'].iloc[0] + addition_range
        # low_level = current_week_data['open'].iloc[0] - addition_range
        # print(current_week)
        # print("High Level:\n", high_level, "\nLow Level:\n", low_level)

        breakout_high = current_day_data[current_day_data["high"] >= high_level]
        breakout_low = current_day_data[current_day_data["low"] <= low_level]

        atm_strike = int(round(current_day_data["open"].iloc[0] / 50) * 50)

        # break

        if not breakout_high.empty:
            # print('Breakout High')
            breakout_high_time = breakout_high.iloc[0]["datetime"]
            # print('High Break At :', breakout_high_time)
            # atm_strike = int(round(low_level / 50) * 50)
            # exit_date = current_week['datetime'].date() + dt.timedelta(days=5)
            expiry = await get_expiry_nifty(current_day["datetime"].date())
            dte = (expiry - breakout_high_time.date()).days
            contract = await get_option_contract_name(
                symbol=index_name,
                strike=atm_strike,
                expiry=expiry,
                opt_type="PE",
            )
            
            # Exit Scenario
            current_day_data_after_entry = current_day_data[
                (current_day_data["datetime"] > breakout_high_time)
                & (current_day_data["datetime"].dt.time <= dt.time(15, 30))
            ]
            
            pe_df = await fetch_data(
                index=index_str_for_opt,
                start_date=breakout_high_time.date(),
                end_date=breakout_high_time.date(),
                start_time=breakout_high_time.time(),
                end_time=dt.time(15, 20),
                expiry=expiry,
                strike=atm_strike,
                asset_class="P",
            )
            if not isinstance(pe_df, str) and pe_df is not None:
                pe_df = pe_df.to_pandas()
                entry_price = pe_df.iloc[0]["c"]
                pe_df_after_entry = pe_df[
                    (pe_df["datetime"] > breakout_high_time)
                    & (pe_df["datetime"].dt.time <= dt.time(15, 30))
                ]
                highest_high_value = pe_df_after_entry['h'].cummax()
                lowest_low_value = pe_df_after_entry['l'].cummin()
                if not highest_high_value.empty:
                    max_high = highest_high_value.max()
                else:
                    max_high = float("nan")
                if not lowest_low_value.empty:
                    max_low = lowest_low_value.min()
                else:
                    max_low = float("nan")
                # print(highest_high_value, lowest_low_value)
            else:
                entry_price = float("nan")
                highest_high_value = float("nan")
                lowest_low_value = float("nan")

            # print(current_week_data_after_entry)
            low_breach = current_day_data_after_entry[
                current_day_data_after_entry["low"] < previous_day_low
            ]
            # print(low_breach)
            if len(low_breach) != 0:
                exit_time = low_breach.iloc[0]["datetime"]
                # print(exit_time)
                remark = "SL Hit"
                # print(remark, exit_time)
                # print(pe_df)
                if not math.isnan(entry_price):
                    # print((pe_df[(pe_df['datetime'].dt.date == exit_time.date()) & (pe_df['datetime'].dt.hour >= exit_time.hour) & (pe_df['datetime'].dt.minute >= exit_time.minute)]).head(3))
                    filtered_df = pe_df.loc[
                        (pe_df["datetime"].dt.date >= exit_time.date())
                        & (pe_df["datetime"].dt.hour >= exit_time.hour)
                        & (pe_df["datetime"].dt.minute >= exit_time.minute),
                        "c"]
                    exit_price = filtered_df.iloc[0] if not filtered_df.empty else float('nan')

                else:
                    exit_price = float("nan")
                    # print(exit_price)
            else:
                exit_time = dt.datetime.combine(
                    current_day["datetime"].date(), dt.time(15, 30)
                )
                remark = "EOD Closing"
                # print(remark, exit_time)
                if not math.isnan(entry_price):
                    exit_price = pe_df.iloc[-1]["c"]
                else:
                    exit_price = float("nan")
                    # print(exit_price)
            # print(exit_time)
            # print(exit_price)
            # print(contract)
            qty = portfolio_value * index_leverage / atm_strike
            slippage = 0.01 * (entry_price + exit_price)
            pnl = qty * (entry_price - exit_price)
            final_pnl = qty * (entry_price - exit_price - slippage)
            unit = {
                "Previous Day": previous_day["datetime"].date(),
                "Day High": previous_day_high,
                "Day Low": previous_day_low,
                "Daily Range": daily_range,
                "Current Day Open": current_day_open,
                "Multiplier to Range": f"{int(multiplier_to_range * 100)}%",
                "Entry Level Long": high_level,
                "Entry Level Short": low_level,
                "Break Type": "high",
                "Strike": atm_strike,
                "Expiry": expiry,
                "DTE": dte,
                "Contract": contract,
                "Option Type": contract[-2:],
                "Entry Time": breakout_high_time,
                "Entry Price": entry_price,
                "Exit Time": exit_time,
                "Exit Price": exit_price,
                "Max High": max_high,
                "Min Low": max_low,
                "Remark": remark,
                "Points": entry_price - exit_price,
                "Slippage": slippage,
                "Points w cs": (entry_price - exit_price) - slippage,
                "Qty": qty,
                "PnL": pnl,
                "PnL w cs": final_pnl,
                "ROI%": (pnl * 100 / portfolio_value),
                "ROI% w cs": (final_pnl * 100 / portfolio_value),
            }
            # print(unit)
            results.append(unit)

        if not breakout_low.empty:
            # print('Breakout Low')
            breakout_low_time = breakout_low.iloc[0]["datetime"]
            # print('Low Break At :', breakout_low_time)
            # atm_strike = int(round(high_level / 50) * 50)
            # exit_date = current_week['datetime'].date() + dt.timedelta(days=5)
            expiry = await get_expiry_nifty(current_day["datetime"].date())
            dte = (expiry - breakout_low_time.date()).days
            contract = await get_option_contract_name(
                symbol=index_name,
                strike=atm_strike,
                expiry=expiry,
                opt_type="CE",
            )
            
            # Exit Scenario
            current_day_data_after_entry = current_day_data[
                (current_day_data["datetime"] > breakout_low_time)
                & (current_day_data["datetime"].dt.time <= dt.time(15, 30))
            ]
            
            ce_df = await fetch_data(
                index=index_str_for_opt,
                start_date=breakout_low_time.date(),
                end_date=breakout_low_time.date(),
                start_time=breakout_low_time.time(),
                end_time=dt.time(15, 20),
                expiry=expiry,
                strike=atm_strike,
                asset_class="C",
            )
            # print(len(ce_df))
            if not isinstance(ce_df, str) and ce_df is not None:
                ce_df = ce_df.to_pandas()
                entry_price = ce_df.iloc[0]["c"]
                ce_df_after_entry = ce_df[
                    (ce_df["datetime"] > breakout_low_time)
                    & (ce_df["datetime"].dt.time <= dt.time(15, 30))
                ]
                highest_high_value = ce_df_after_entry['h'].cummax()
                lowest_low_value = ce_df_after_entry['l'].cummin()
                if not highest_high_value.empty:
                    max_high = highest_high_value.max()
                else:
                    max_high = float("nan")
                if not lowest_low_value.empty:
                    max_low = lowest_low_value.min()
                else:
                    max_low = float("nan")
            else:
                entry_price = float("nan")
                highest_high_value = float("nan")
                lowest_low_value = float("nan")
            
            # print(current_week_data_after_entry)
            high_breach = current_day_data_after_entry[
                current_day_data_after_entry["high"] > previous_day_high
            ]
            # print(high_breach)
            if len(high_breach) != 0:
                exit_time = high_breach.iloc[0]["datetime"]
                # print(exit_time)
                remark = "SL Hit"
                # print(remark, exit_time)
                if not math.isnan(entry_price):
                    # print(exit_time.date(), exit_time.hour, exit_time.minute)
                    # print((ce_df[(ce_df['datetime'].dt.date == exit_time.date()) & (ce_df['datetime'].dt.hour >= exit_time.hour) & (ce_df['datetime'].dt.minute >= exit_time.minute)]).head(3))
                    filtered_df = ce_df.loc[
                        (ce_df["datetime"].dt.date >= exit_time.date())
                        & (ce_df["datetime"].dt.hour >= exit_time.hour)
                        & (ce_df["datetime"].dt.minute >= exit_time.minute),
                        "c"]
                    exit_price = filtered_df.iloc[0] if not filtered_df.empty else float('nan') 
                else:
                    exit_price = float("nan")
                    # print(exit_price)
            else:
                exit_time = dt.datetime.combine(
                    current_day["datetime"].date(), dt.time(15, 30)
                )
                remark = "EOD Closing"
                # print(remark, exit_time)
                if not math.isnan(entry_price):
                    exit_price = ce_df.iloc[-1]["c"]
                else:
                    exit_price = float("nan")
                    # print(exit_price)
            # print(exit_time)
            # print(exit_price)
            # print(contract)

            qty = portfolio_value * index_leverage / atm_strike
            slippage = 0.01 * (entry_price + exit_price)
            pnl = qty * (entry_price - exit_price)
            final_pnl = qty * (entry_price - exit_price - slippage)
            unit = {
                "Previous Day": previous_day["datetime"].date(),
                "Day High": previous_day_high,
                "Day Low": previous_day_low,
                "Daily Range": daily_range,
                "Current Day Open": current_day_open,
                "Multiplier to Range": f"{int(multiplier_to_range * 100)}%",
                "Entry Level Long": high_level,
                "Entry Level Short": low_level,
                "Break Type": "low",
                "Strike": atm_strike,
                "Expiry": expiry,
                "DTE": dte,
                "Contract": contract,
                "Option Type": contract[-2:],
                "Entry Time": breakout_low_time,
                "Entry Price": entry_price,
                "Exit Time": exit_time,
                "Exit Price": exit_price,
                "Max High": max_high,
                "Min Low": max_low,
                "Remark": remark,
                "Points": entry_price - exit_price,
                "Slippage": slippage,
                "Points w cs": (entry_price - exit_price) - slippage,
                "Qty": qty,
                "PnL": pnl,
                "PnL w cs": final_pnl,
                "ROI%": (pnl * 100 / portfolio_value),
                "ROI% w cs": (final_pnl * 100 / portfolio_value),
            }
            # print(unit)
            results.append(unit)

    # print(results)
    return results

In [10]:
async def trade(n, pct):
    df = bnf_1min
    df = df.rename(columns={
        "o": "open",
        "h": "high",
        "l": "low",
        "c": "close"
    })
    # print(df.tail())
    # weekly_ranges = calculate_weekly_ranges(df)
    # weekly_ranges = resample(pl.DataFrame(df), "7d", pd.Timedelta(days=1))
    resample_days = f'{n}d'
    daily_ranges = resample(pl.DataFrame(df), resample_days)
    daily_ranges = daily_ranges.to_pandas()
    # print(daily_ranges.tail())
    breakouts = await find_breakouts_v2(df, daily_ranges, pct)
    breakouts_pandas = pd.DataFrame(breakouts)
    # breakouts_polars = pl.DataFrame(breakouts)
    return breakouts_pandas
    # return

In [17]:
tradebook = await trade(1, 1.25)
# tradebook

Current day :  2017-01-03
Current day :  2017-01-04
Current day :  2017-01-05
Current day :  2017-01-06
Current day :  2017-01-09
Current day :  2017-01-10
Current day :  2017-01-11
Current day :  2017-01-12
Current day :  2017-01-13
Current day :  2017-01-16
Current day :  2017-01-17
Current day :  2017-01-18
Current day :  2017-01-19
Current day :  2017-01-20
Current day :  2017-01-23
Current day :  2017-01-24
Current day :  2017-01-25
Current day :  2017-01-27
Current day :  2017-01-30
Current day :  2017-01-31
Current day :  2017-02-01
Current day :  2017-02-02
Current day :  2017-02-03
Current day :  2017-02-06
Current day :  2017-02-07
Current day :  2017-02-08
Current day :  2017-02-09
Current day :  2017-02-10
Current day :  2017-02-13
Current day :  2017-02-14
Current day :  2017-02-15
Current day :  2017-02-16
Current day :  2017-02-17
Current day :  2017-02-20
Current day :  2017-02-21
Current day :  2017-02-22
Current day :  2017-02-23
Current day :  2017-02-27
Current day 

In [18]:
tradebook

,Previous Day,Day High,Day Low,Daily Range,Current Day Open,Multiplier to Range,Entry Level Long,Entry Level Short,Break Type,Strike,Expiry,DTE,Contract,Option Type,Entry Time,Entry Price,Exit Time,Exit Price,Max High,Min Low,Remark,Points,Slippage,Points w cs,Qty,PnL,PnL w cs,ROI%,ROI% w cs
0,2017-01-04,8218.4500,8180.9500,37.5000,8226.6500,125%,8273.5250,8179.7750,high,8250,2017-01-05,0,NIFTY171058250PE,PE,2017-01-05 13:55:00,NaN,2017-01-05 15:30:00,NaN,NaN,NaN,EOD Closing,NaN,NaN,NaN,9818.1818,NaN,NaN,NaN,NaN
1,2017-01-10,8293.5500,8262.2500,31.3000,8329.8500,125%,8368.9750,8290.7250,high,8350,2017-01-12,1,NIFTY171128350PE,PE,2017-01-11 13:35:00,NaN,2017-01-11 15:30:00,NaN,NaN,NaN,EOD Closing,NaN,NaN,NaN,9700.5988,NaN,NaN,NaN,NaN
2,2017-01-12,8417.0000,8382.3500,34.6500,8457.6500,125%,8500.9625,8414.3375,low,8450,2017-01-19,6,NIFTY171198450CE,CE,2017-01-13 09:30:00,NaN,2017-01-13 09:31:00,NaN,NaN,NaN,SL Hit,NaN,NaN,NaN,9585.7988,NaN,NaN,NaN,NaN
3,2017-01-19,8445.0500,8404.3000,40.7500,8404.3500,125%,8455.2875,8353.4125,low,8400,2017-01-25,5,NIFTY17JAN8400CE,CE,2017-01-20 13:56:00,30.4000,2017-01-20 15:30:00,28.2000,32.0000,26.0500,EOD Closing,2.2000,0.5860,1.6140,9642.8571,21214.2857,15563.5714,0.2357,0.1729
4,2017-01-24,8480.7500,8399.4500,81.3000,8499.4500,125%,8601.0750,8397.8250,high,8500,2017-01-25,0,NIFTY17JAN8500PE,PE,2017-01-25 15:07:00,0.0500,2017-01-25 15:30:00,0.0500,0.0500,0.0500,EOD Closing,0.0000,0.0010,-0.0010,9529.4118,0.0000,-9.5294,0.0000,-0.0001
5,2017-01-30,8662.5500,8617.8000,44.7500,8629.4500,125%,8685.3875,8573.5125,low,8650,2017-02-02,2,NIFTY172028650CE,CE,2017-01-31 11:31:00,NaN,2017-01-31 15:30:00,NaN,0.0500,0.0500,EOD Closing,NaN,NaN,NaN,9364.1618,NaN,NaN,NaN,NaN
6,2017-01-31,8629.4500,8552.8500,76.6000,8570.3500,125%,8666.1000,8474.6000,high,8550,2017-02-02,1,NIFTY172028550PE,PE,2017-02-01 13:50:00,NaN,2017-02-01 15:30:00,NaN,0.0500,0.0500,EOD Closing,NaN,NaN,NaN,9473.6842,NaN,NaN,NaN,NaN
7,2017-02-06,8814.0000,8770.2000,43.8000,8805.7000,125%,8860.4500,8750.9500,low,8800,2017-02-09,2,NIFTY172098800CE,CE,2017-02-07 14:40:00,NaN,2017-02-07 15:30:00,NaN,0.0500,0.0500,EOD Closing,NaN,NaN,NaN,9204.5455,NaN,NaN,NaN,NaN
8,2017-02-10,8821.6500,8771.2000,50.4500,8818.8500,125%,8881.9125,8755.7875,low,8800,2017-02-16,3,NIFTY172168800CE,CE,2017-02-13 12:54:00,NaN,2017-02-13 15:30:00,NaN,0.0500,0.0500,EOD Closing,NaN,NaN,NaN,9204.5455,NaN,NaN,NaN,NaN
9,2017-02-14,8819.9000,8772.6500,47.2500,8777.6500,125%,8836.7125,8718.5875,low,8800,2017-02-16,1,NIFTY172168800CE,CE,2017-02-15 12:41:00,NaN,2017-02-15 15:30:00,NaN,0.0500,0.0500,EOD Closing,NaN,NaN,NaN,9204.5455,NaN,NaN,NaN,NaN


In [19]:
tradebook['ROI% w cs'].sum()

45.89072637562323

In [20]:
# x = tradebook['Entry Time'].iloc[0].year
tradebook["Entry Time"] = pd.to_datetime(tradebook["Entry Time"])
tradebook["Trade Year"] = tradebook["Entry Time"].dt.year

In [21]:
def generate_stats(tb, variation):
    stats_df8 = pd.DataFrame(
        index=range(2017, 2025),
        columns=[
            "Total ROI",
            "Total Trades",
            "Win Rate",
            "Avg Profit% per Trade",
            "Avg Loss% per Trade",
            "Max Drawdown",
            "ROI/DD Ratio",
            "Variation",
        ],
    )
    combined_df_sorted = tb
    # combined_df_sorted = tb_expiry_ce
    # combined_df_sorted = tb_expiry_pe
    
    # Iterate over each year
    for year in range(2017, 2025):
        # Filter trades for the current year
        year_trades = combined_df_sorted[(combined_df_sorted["Trade Year"] == year)]
    
        # Calculate total ROI
        total_roi = year_trades["ROI% w cs"].sum()
    
        # Calculate total number of trades
        total_trades = len(year_trades)
    
        # Calculate win rate
        win_rate = (year_trades["ROI% w cs"] > 0).mean() * 100
    
        # Calculate average profit per trade
        avg_profit = year_trades[year_trades["ROI% w cs"] > 0]["ROI% w cs"].mean()
    
        # Calculate average loss per trade
        avg_loss = year_trades[year_trades["ROI% w cs"] < 0]["ROI% w cs"].mean()
    
        # Calculate maximum drawdown
        max_drawdown = (
            year_trades["ROI% w cs"].cumsum() - year_trades["ROI% w cs"].cumsum().cummax()
        ).min()
    
        # Calculate ROI/DD ratio
        roi_dd_ratio = total_roi / abs(max_drawdown)

        variation = variation
    
        # Store the statistics in the DataFrame
        stats_df8.loc[year] = [
            total_roi,
            total_trades,
            win_rate,
            avg_profit,
            avg_loss,
            max_drawdown,
            roi_dd_ratio,
            variation,
        ]
    
    # Calculate overall statistics
    overall_total_roi = stats_df8["Total ROI"].sum()
    overall_total_trades = stats_df8["Total Trades"].sum()
    overall_win_rate = (combined_df_sorted["ROI% w cs"] > 0).mean() * 100
    overall_avg_profit = combined_df_sorted[combined_df_sorted["ROI% w cs"] > 0]["ROI% w cs"].mean()
    overall_avg_loss = combined_df_sorted[combined_df_sorted["ROI% w cs"] < 0]["ROI% w cs"].mean()
    overall_max_drawdown = (
        combined_df_sorted["ROI% w cs"].cumsum() - combined_df_sorted["ROI% w cs"].cumsum().cummax()
    ).min()
    overall_roi_dd_ratio = overall_total_roi / abs(overall_max_drawdown)
    overall_variation = variation

    
    # Store the overall statistics in the DataFrame
    stats_df8.loc["Overall"] = [
        overall_total_roi,
        overall_total_trades,
        overall_win_rate,
        overall_avg_profit,
        overall_avg_loss,
        overall_max_drawdown,
        overall_roi_dd_ratio,
        overall_variation,
    ]
    
    # print(f'{overall_total_roi} , {overall_max_drawdown} , {overall_roi_dd_ratio}')
    
    return {overall_roi_dd_ratio: stats_df8}

In [22]:
stats = generate_stats(tradebook, '...')
for x, y in stats.items():
    pdx = pd.DataFrame(y)
    break

pdx

,Total ROI,Total Trades,Win Rate,Avg Profit% per Trade,Avg Loss% per Trade,Max Drawdown,ROI/DD Ratio,Variation
2017,1.1231,60,13.3333,0.2899,-0.2393,-1.0769,1.0429,...
2018,1.2999,57,15.7895,0.5339,-0.8762,-2.4639,0.5276,...
2019,5.5487,62,53.2258,0.8117,-0.9654,-15.0495,0.3687,...
2020,11.8509,54,61.1111,1.0813,-1.1349,-10.1354,1.1693,...
2021,2.2352,62,56.4516,0.6341,-0.8316,-8.9199,0.2506,...
2022,7.3637,49,65.3061,0.7209,-0.9238,-7.7001,0.9563,...
2023,4.7964,43,62.7907,0.4700,-0.5638,-3.4419,1.3935,...
2024,11.6729,66,66.6667,0.6948,-0.8999,-7.6716,1.5216,...
Overall,45.8907,453,48.7859,0.7155,-0.8768,-15.0495,3.0493,...


In [24]:
tradebook.to_csv('RB-Daily_1_1pt25.csv', index=False)

In [120]:
stats_dictionary = {}
pct_range = [0.1, 0.2, 0.3, 0.4, 0.5, 0.66, 0.75, 0.9, 1, 1.25, 1.5]

for i in range(1, 8):         # No. of days for range
    for j in pct_range:
        variation = f'Days: {i}, PCT: {j}'
        print(variation)
        tb = await trade(i, j)
        if len(tb) > 0:
            tb["Entry Time"] = pd.to_datetime(tb["Entry Time"])
            tb["Trade Year"] = tb["Entry Time"].dt.year
            stats = generate_stats(tb, variation)
            for x, y in stats.items():
                if x>1:
                    print(y.to_string())
                    stats_dictionary[x] = y

Days: 1, PCT: 0.1
Days: 1, PCT: 0.2
Days: 1, PCT: 0.3
Days: 1, PCT: 0.4
Days: 1, PCT: 0.5
        Total ROI Total Trades Win Rate Avg Profit% per Trade Avg Loss% per Trade Max Drawdown ROI/DD Ratio          Variation
2017       0.7427          211  13.7441                0.4188             -0.5701      -4.1484       0.1790  Days: 1, PCT: 0.5
2018       3.8801          214  15.4206                0.6599             -0.9942      -5.2418       0.7402  Days: 1, PCT: 0.5
2019      -5.1825          198  57.5758                0.6475             -1.2538     -17.1896      -0.3015  Days: 1, PCT: 0.5
2020      -1.6319          202  55.4455                1.1307             -1.5271     -40.2642      -0.0405  Days: 1, PCT: 0.5
2021      35.0404          201  60.1990                0.6978             -0.6767     -10.8675       3.2243  Days: 1, PCT: 0.5
2022      12.0154          217  55.7604                0.7467             -0.8246     -12.0166       0.9999  Days: 1, PCT: 0.5
2023      -5.7753    

/tmp/ipykernel_2432/2776817619.py:45: RuntimeWarning: divide by zero encountered in scalar divide
  roi_dd_ratio = total_roi / abs(max_drawdown)


        Total ROI Total Trades Win Rate Avg Profit% per Trade Avg Loss% per Trade Max Drawdown ROI/DD Ratio          Variation
2017      -0.7028           39  12.8205                0.0668             -0.2592      -0.9031      -0.7782  Days: 1, PCT: 1.5
2018       1.4568           35  14.2857                0.2966             -0.0265       0.0000          inf  Days: 1, PCT: 1.5
2019       8.4452           42  59.5238                0.4489             -0.1984      -1.9140       4.4123  Days: 1, PCT: 1.5
2020       1.4742           40  62.5000                0.4158             -0.5946      -6.1537       0.2396  Days: 1, PCT: 1.5
2021      -0.4050           39  64.1026                0.3286             -0.7184      -6.5280      -0.0620  Days: 1, PCT: 1.5
2022       0.8303           35  68.5714                0.2765             -0.5278      -3.7846       0.2194  Days: 1, PCT: 1.5
2023       4.2452           26  61.5385                0.2965             -0.0499      -0.1770      23.9864  Da

IndexError: single positional indexer is out-of-bounds